In [ ]:
!pip install textstat -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import models, transforms
from transformers import BertTokenizer, VisualBertForPreTraining
from PIL import Image
from tqdm import tqdm
import requests
from io import BytesIO
import json
import os
import random
import gc
import h5py
import pickle
import numpy as np
import time


import argparse
import textstat
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from collections import Counter

In [ ]:
seed = 42

nlp = spacy.load("en_core_web_sm")

stop_words = STOP_WORDS

# Feature extraction function
def extract_nlp_features(text):
    # Process the text with spaCy. This includes tokenization, lemmatization, POS tagging, and NER
    doc = nlp(text)

    # Calculate Type-Token Ratio (TTR) for lexical diversity
    tokens = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]
    unique_words = set(tokens)
    ttr = len(unique_words) / len(tokens) if tokens else 0

    # Sentence complexity (average number of words per sentence)
    sentences = list(doc.sents)
    sentence_complexity = sum(len(sentence) for sentence in sentences) / len(sentences) if sentences else 0

    # Readability score using Flesch-Kincaid grade level
    readability_score = textstat.flesch_kincaid_grade(text)

    # POS distribution
    pos_dist = Counter(token.pos_ for token in doc)

    # Named Entity Recognition using spaCy
    ner_entities = [ent.text for ent in doc.ents]
    ner_freq_dist = Counter(ner_entities)

    # Merge the dictionaries
    features = {
        'type_token_ratio': ttr,
        'sentence_complexity': sentence_complexity,
        'readability_score': readability_score,
        'pos_distribution': dict(pos_dist),
        'named_entity_frequency': dict(ner_freq_dist)
    }
    return features

def request_image(url, transform):
    rsp = requests.get(url, stream=True)
    img = Image.open(BytesIO(rsp.content)).convert('RGB')
    return transform(img)

def load_csv_dataset(file_path, image_size=(224, 224)):
    df = pd.read_csv(file_path, nrows=2400)
    transform = transforms.Compose([transforms.Resize(image_size), transforms.ToTensor()])
    processed_data = []

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        story1 = row['sent1']
        story2 = row['sent2']
        label = row['label']

        # Parse the URL field which is in string representation of list
        urls = json.loads(row['url'].replace("'", '"'))

        img_tensors = []
        for url in urls:
            try:
                img_tensors.append(request_image(url, transform))
            except Exception as e:
                # print(f"Error downloading image: {e}")
                continue

        if len(img_tensors) == 0:
            continue

        img_tensor = torch.stack(img_tensors)

        processed_data.append({'image': img_tensor, 'story1': story1, 'story2': story2, 'label': label})

    return processed_data

def collate_fn(batch):
    # Pad the visual embeddings to have the same sequence length
    image_features = pad_sequence([item['visual_embeds'] for item in batch], batch_first=True)

    # Get the maximum sequence length for visual_attention_mask
    max_seq_length = max(item['visual_attention_mask'].shape[1] for item in batch)

    # Pad the visual_attention_mask tensors
    visual_attention_mask = [torch.cat([item['visual_attention_mask'], torch.zeros(item['visual_attention_mask'].shape[0], max_seq_length - item['visual_attention_mask'].shape[1])], dim=1) for item in batch]

    # Similarly, you can pad other sequences if necessary.
    # Stack other values
    input_ids_1 = torch.stack([item['input_ids_1'] for item in batch])
    attention_mask_1 = torch.stack([item['attention_mask_1'] for item in batch])
    visual_attention_mask = torch.stack(visual_attention_mask)
    ranker_gap = torch.tensor([item['ranker_gap'] for item in batch])

    return {
        'input_ids_1': input_ids_1,
        'attention_mask_1': attention_mask_1,
        'visual_embeds': image_features,
        'visual_attention_mask': visual_attention_mask,
        'ranker_gap': ranker_gap
    }


# Custom Dataset class
class RankingDataset(Dataset):
    def __init__(self, data, tokenizer, resnet_model, max_length=512, mask_probability=0.15):
        self.data = data
        self.tokenizer = tokenizer
        self.resnet_model = resnet_model
        self.max_length = max_length
        self.mask_probability = mask_probability

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Process image
        image_feature_list = []
        # Extract features using ResNet
        for image in item['image']:
            with torch.no_grad():
                # Inside the __getitem__ method
                image_features = self.resnet_model(image.unsqueeze(0))
                # print("Shape after ResNet:", image_features.shape)

                image_features = image_features.view(image_features.size(0), -1)
                # print("Shape after flattening:", image_features.shape)

                image_feature_list.append(image_features)
                #image_features = self.projection(image_features)
                #print("Shape after projection:", image_features.shape)

        image_features = torch.cat(image_feature_list, dim=0)

        # Process text
        story1 = item['story1']
        story2 = item['story2']

        # Extract features
        features1 = extract_nlp_features(story1)
        features2 = extract_nlp_features(story2)

        # Create combined text with features
        combined_story1 = self.create_feature_text(story1, features1)
        combined_story2 = self.create_feature_text(story2, features2)

        combined_input = combined_story1 + ' [SEP] ' + combined_story2

        # Tokenize text and prepare inputs for VisualBERT
        inputs_1 = self.tokenizer.encode_plus(
            combined_input,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        inputs_ids_1 = inputs_1['input_ids'].squeeze()

        attention_mask_1 = inputs_1['attention_mask']

        visual_attention_mask = torch.ones((image_features.size(0),), dtype=torch.long).unsqueeze(0)

        ranker_gap = item['label']

        return {
            'input_ids_1': inputs_ids_1,
            'attention_mask_1': attention_mask_1,
            'visual_embeds': image_features,
            'visual_attention_mask': visual_attention_mask,
            'ranker_gap': ranker_gap
        }

    def create_feature_text(self, story_text, features):
        # Format and append the features to the story text
        feature_text = f"{story_text} | TTR: {features['type_token_ratio']} | Complexity: {features['sentence_complexity']} | Readability: {features['readability_score']} | POS: {json.dumps(features['pos_distribution'])}"
        return feature_text

# Initialize components
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
resnet_model = models.resnet50(pretrained=True)
resnet_model = nn.Sequential(*list(resnet_model.children())[:-1])
resnet_model.eval()

print("Components initialized.")

# Load the dataset
print("Loading dataset...")

start = time.time()

loaded_data = []

with h5py.File('/content/drive/MyDrive/Colab Notebooks/project/VHED_url.hdf5', 'r') as f:
    for group_key in f:
        group = f[group_key]
        item = {}
        for key in group:
            # Read the pickled data
            pickled_data = group[key][()].tobytes()
            # Deserialize using pickle
            item[key] = pickle.loads(pickled_data)
        loaded_data.append(item)

end = time.time()

print(f'Loaded {len(loaded_data)} items in {end - start} seconds')

# ranking_data = load_csv_dataset('/content/drive/MyDrive/Colab Notebooks/project/VHED_url.csv')
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ind = int(len(ranking_data) * 0.2)
# sub_loaded_data = ranking_data[:ind]
sub_loaded_data = loaded_data

random.seed(seed)

random.shuffle(sub_loaded_data)

train_data_ind = int(0.6 * len(sub_loaded_data))
val_data_ind = int(0.8 * len(sub_loaded_data))

train_data = sub_loaded_data[:train_data_ind]
val_data = sub_loaded_data[train_data_ind:val_data_ind]
test_data = sub_loaded_data[val_data_ind:]

train_dataset = RankingDataset(train_data, tokenizer, resnet_model)
val_dataset = RankingDataset(val_data, tokenizer, resnet_model)
test_dataset = RankingDataset(test_data, tokenizer, resnet_model)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=8)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=8)

# Training loop
def train_ranker(model, dataloader, optimizer, criterion):
    model.train()

    total_correct = 0
    total_sample = 0

    for batch in tqdm(dataloader):
        input_ids_1 = batch['input_ids_1']
        attention_mask_1 = batch['attention_mask_1']
        visual_embeds = batch['visual_embeds']
        visual_attention_mask = batch['visual_attention_mask']
        ranking_gap = batch['ranker_gap'].float()

        # move everything to device
        input_ids_1 = input_ids_1.to(DEVICE)
        attention_mask_1 = attention_mask_1.to(DEVICE)
        visual_embeds = visual_embeds.to(DEVICE)
        visual_attention_mask = visual_attention_mask.to(DEVICE)
        ranking_gap = ranking_gap.to(DEVICE)

        # Forward pass and loss computation for both story inputs
        outputs_1 = model(
            input_ids=input_ids_1,
            visual_embeds=visual_embeds,
            attention_mask=attention_mask_1,  # Add this
            visual_attention_mask=visual_attention_mask,  # Add this
            output_hidden_states=True
        )

        outputs_1 = model.dropout(outputs_1.hidden_states[-1][:, 0])

        # Use the new ranking layer to predict the ranking gap
        ranking_prediction_1 = model.regression_layer(outputs_1)
        ranking_prediction = ranking_prediction_1.squeeze(-1)

        # compute accuracy
        for i in range(len(ranking_prediction)):
            if ranking_prediction[i] > 0 and ranking_gap[i] > 0:
                total_correct += 1
            elif ranking_prediction[i] < 0 and ranking_gap[i] < 0:
                total_correct += 1

        total_sample += ranking_prediction.shape[0]

        ranking_prediction = ranking_prediction.to(DEVICE)

        # Compute loss
        loss = criterion(ranking_prediction, ranking_gap)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Train Accuracy: {total_correct / total_sample}")
    print(f"Train Loss: {loss.item()}")

    torch.cuda.empty_cache()
    gc.collect()

def evaluate_ranker(model, dataloader, criterion):
    model.eval()
    total_correct = 0
    total_sample = 0
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids_1 = batch['input_ids_1']
            attention_mask_1 = batch['attention_mask_1']
            visual_embeds = batch['visual_embeds']
            visual_attention_mask = batch['visual_attention_mask']
            ranking_gap = batch['ranker_gap'].float()

            #
            input_ids_1 = input_ids_1.to(DEVICE)
            attention_mask_1 = attention_mask_1.to(DEVICE)
            visual_embeds = visual_embeds.to(DEVICE)
            visual_attention_mask = visual_attention_mask.to(DEVICE)
            ranking_gap = ranking_gap.to(DEVICE)

            # Forward pass
            outputs1 = model(
                input_ids=input_ids_1,
                visual_embeds=visual_embeds,
                attention_mask=attention_mask_1,  # Add this
                visual_attention_mask=visual_attention_mask,
                output_hidden_states=True
            )

            #
            ranking_prediction_1 = model.regression_layer(outputs1.hidden_states[-1][:, 0])
            ranking_prediction = ranking_prediction_1.squeeze(-1)

            ranking_prediction = ranking_prediction.to(DEVICE)
            #
            loss = criterion(ranking_prediction, ranking_gap)
            total_loss += loss.item()
            #
            for i in range(len(ranking_prediction)):
                if ranking_prediction[i] > 0 and ranking_gap[i] > 0:
                    total_correct += 1

                elif ranking_prediction[i] < 0 and ranking_gap[i] < 0:
                    total_correct += 1
            total_sample += ranking_prediction.shape[0]
    print(f"Val Accuracy: {total_correct / total_sample}")
    print(f"Val Loss: {total_loss / total_sample}")

    return total_correct / total_sample

# Load your pre-trained VisualBert model
model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
model.load_state_dict(torch.load('/content/drive/MyDrive/Project/VIST-Inspector-main/model/epoch_1.pt'))
model.eval()  # Make sure the model is in evaluation mode

# Modify model for ranking task
# Add a linear layer to model for regression task
model.regression_layer = nn.Linear(model.config.hidden_size, 1)

model.dropout = nn.Dropout(p=0.1)

# Define optimizer and loss function for ranking
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-5)
criterion = nn.MSELoss()

model = model.to(DEVICE)

In [ ]:
num_epochs = 10

save_path = '/content/drive/MyDrive/Colab Notebooks/project/subset'

best_val_acc = 0

for epoch in range(3, num_epochs):
    train_ranker(model, train_dataloader, optimizer, criterion)
    val_acc = evaluate_ranker(model, val_dataloader, criterion)
    print(f"Epoch {epoch} complete.")
    # Save the fine-tuned model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_path, f'ranker_pair_best_epoch_{epoch}.pt'))

In [ ]:
def test_ranker(model, dataloader, criterion):
    model.eval()
    total_correct = 0
    total_sample = 0
    total_loss = 0
    results = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids_1 = batch['input_ids_1']
            attention_mask_1 = batch['attention_mask_1']
            visual_embeds = batch['visual_embeds']
            visual_attention_mask = batch['visual_attention_mask']
            ranking_gap = batch['ranker_gap'].float()

            #
            input_ids_1 = input_ids_1.to(DEVICE)
            attention_mask_1 = attention_mask_1.to(DEVICE)
            visual_embeds = visual_embeds.to(DEVICE)
            visual_attention_mask = visual_attention_mask.to(DEVICE)
            ranking_gap = ranking_gap.to(DEVICE)

            # Forward pass
            outputs1 = model(
                input_ids=input_ids_1,
                visual_embeds=visual_embeds,
                attention_mask=attention_mask_1,  # Add this
                visual_attention_mask=visual_attention_mask,
                output_hidden_states=True
            )
            #
            ranking_prediction_1 = model.regression_layer(outputs1.hidden_states[-1][:, 0])
            ranking_prediction = ranking_prediction_1.squeeze(-1)

            results.extend(ranking_prediction.tolist())

            ranking_prediction = ranking_prediction.to(DEVICE)
            #
            loss = criterion(ranking_prediction, ranking_gap)
            total_loss += loss.item()
            #
            for i in range(len(ranking_prediction)):
                if ranking_prediction[i] > 0 and ranking_gap[i] > 0:
                    total_correct += 1

                elif ranking_prediction[i] < 0 and ranking_gap[i] < 0:
                    total_correct += 1
            total_sample += ranking_prediction.shape[0]
    print(f"Test Accuracy: {total_correct / total_sample}")
    print(f"Test Loss: {total_loss / total_sample}")

    return results

In [ ]:
finetuned_model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-vqa-coco-pre')

finetuned_model.regression_layer = nn.Linear(finetuned_model.config.hidden_size, 1)

finetuned_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/project/subset/ranker_pair_best_epoch_4.pt'))

finetuned_model = finetuned_model.to(DEVICE)

In [ ]:
results = test_ranker(finetuned_model, test_dataloader, criterion)

100%|██████████| 632/632 [08:21<00:00,  1.26it/s]

Test Accuracy: 0.8828650573802929
Test Loss: 0.11451821406101008


In [ ]:
import csv

# write test dataloader column story1, story2, label and the above reults to csv
with open('/content/drive/MyDrive/Colab Notebooks/project/subset/ranker_w_ling_pair_test_results.csv', 'w') as f:
    writer = csv.writer(f)

    writer.writerow(['story1', 'story2', 'label', 'prediction'])

    data = test_dataloader.dataset.data

    for i in range(len(data)):
        writer.writerow([data[i]['story1'], data[i]['story2'], data[i]['label'], results[i]])

    f.close()
    print('Done!')

Done!
